In [1]:
import pandas as pd
import os

In [2]:
# SETTINGS
path = '../010-add_coordinates/coordinates/'
filter_df = pd.read_csv('categories.csv', sep = ';', encoding = "ISO-8859-1")

In [3]:
filter_df = filter_df.loc[~filter_df.SELECT.isna(), ['SELECT', 'Geometry Code', 'Layer', 'fclass']]
#filter_df['Geometry Code'] = filter_df['Geometry Code'].str.pad(4, side = 'right', fillchar = '.')
filter_df['Geometry Code'] = filter_df['Geometry Code'].str.strip('0')

In [4]:
granularity = 4 - filter_df['Geometry Code'].str.len()
multiplier = 10 ** granularity

In [5]:
filter_df['Geometry Code'] = filter_df['Geometry Code'].astype(int)

filter_df['low_threshold'] = multiplier * filter_df['Geometry Code']
filter_df['high_threshold'] = multiplier * filter_df['Geometry Code'] + (multiplier - 1)

filter_df['fclass'] = filter_df['fclass'].fillna(filter_df['Layer'])
filter_df['granularity'] = granularity

filter_df = filter_df.sort_values(by=['granularity'])

filter_df

,SELECT,Geometry Code,Layer,fclass,low_threshold,high_threshold,granularity
0,accessibility,2601,money,bank,2601,2601,0
1,accessibility,2602,money,atm,2602,2602,0
2,accessibility,2901,miscpoi,toilet,2901,2901,0
3,accessibility,2902,miscpoi,bench,2902,2902,0
4,accessibility,2903,miscpoi,drinking_water,2903,2903,0
...,...,...,...,...,...,...,...
239,transports,7227,landuse,railway,7227,7227,0
240,transports,5,transport,transport,5000,5999,3
241,transports,51,roads,roads,5100,5199,2
242,transports,52,traffic,traffic,5200,5299,2


In [6]:
file_list = os.listdir(path)

filter_regex = '|'.join(r"{}".format(x) for x in filter_df['Geometry Code'])

In [7]:
df_dict = {}

for file_name in file_list:

    df_dict[file_name] = pd.read_csv(path + file_name)

    df_dict[file_name].drop('Unnamed: 0', axis = 1, inplace = True)

In [8]:
filter_df_to_dict = filter_df[['fclass', 'low_threshold', 'high_threshold']]
filter_df_to_dict.drop_duplicates(subset = ['fclass'], inplace = True)
filter_df_to_dict

C:\Users\gabri\AppData\Local\Temp\ipykernel_33776\544379905.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  filter_df_to_dict.drop_duplicates(subset = ['fclass'], inplace = True)


,fclass,low_threshold,high_threshold
0,bank,2601,2601
1,atm,2602,2602
2,toilet,2901,2901
3,bench,2902,2902
4,drinking_water,2903,2903
...,...,...,...
238,other_lift,6119,6119
239,railway,7227,7227
240,transport,5000,5999
241,roads,5100,5199


In [9]:
filter_df_to_dict.index = filter_df_to_dict.fclass
filter_df_to_dict.drop('fclass', axis = 1, inplace = True)
filter_dict = filter_df_to_dict.to_dict('index')

C:\Users\gabri\AppData\Local\Temp\ipykernel_33776\3101066230.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  filter_df_to_dict.drop('fclass', axis = 1, inplace = True)


In [10]:
filter_df

,SELECT,Geometry Code,Layer,fclass,low_threshold,high_threshold,granularity
0,accessibility,2601,money,bank,2601,2601,0
1,accessibility,2602,money,atm,2602,2602,0
2,accessibility,2901,miscpoi,toilet,2901,2901,0
3,accessibility,2902,miscpoi,bench,2902,2902,0
4,accessibility,2903,miscpoi,drinking_water,2903,2903,0
...,...,...,...,...,...,...,...
239,transports,7227,landuse,railway,7227,7227,0
240,transports,5,transport,transport,5000,5999,3
241,transports,51,roads,roads,5100,5199,2
242,transports,52,traffic,traffic,5200,5299,2


In [11]:
# df = df_dict['coordinates_gis_osm_pois_free_1.csv']
#
# df['fclass'] = 0
#
# for fclass, value_dict in filter_dict.items():
#
#     df.loc[(df.code >= value_dict['low_threshold']) & (df.code <= value_dict['high_threshold']),'fclass'] = fclass
#
#     final_df = df.merge(filter_df[['fclass', 'SELECT']], on = 'fclass', how = 'left')
#
#     final_df = final_df[final_df.fclass != 0]
#
# final_df

In [12]:
filtered_df_dict = {}

for df_name, df in df_dict.items():

    df['fclass'] = 0

    for fclass, value_dict in filter_dict.items():

        df.loc[(df.code >= value_dict['low_threshold']) & (df.code <= value_dict['high_threshold']),'fclass'] = fclass

        final_df = df.merge(filter_df[['fclass', 'SELECT']], on = 'fclass', how = 'left')

        final_df = final_df.drop_duplicates()

        final_df = final_df[final_df.fclass != 0]

    filtered_df_dict[df_name] = final_df

In [13]:
for df_name in filtered_df_dict.keys():
    print('DATAFRAME NAME: ', df_name)
    print('ORIGINAL DATAFRAME SIZE: ', df_dict[df_name].shape)
    print('FILTERED DATAFRAME SIZE: ', filtered_df_dict[df_name].shape)
    print('DIFFERENCE: ', df_dict[df_name].shape[0] - filtered_df_dict[df_name].shape[0])
    print(' ')

DATAFRAME NAME:  coordinates_gis_osm_buildings_a_free_1.csv
ORIGINAL DATAFRAME SIZE:  (4580168, 7)
FILTERED DATAFRAME SIZE:  (0, 8)
DIFFERENCE:  4580168
 
DATAFRAME NAME:  coordinates_gis_osm_landuse_a_free_1.csv
ORIGINAL DATAFRAME SIZE:  (536191, 6)
FILTERED DATAFRAME SIZE:  (394968, 7)
DIFFERENCE:  141223
 
DATAFRAME NAME:  coordinates_gis_osm_natural_a_free_1.csv
ORIGINAL DATAFRAME SIZE:  (1205, 6)
FILTERED DATAFRAME SIZE:  (1205, 7)
DIFFERENCE:  0
 
DATAFRAME NAME:  coordinates_gis_osm_natural_free_1.csv
ORIGINAL DATAFRAME SIZE:  (325604, 6)
FILTERED DATAFRAME SIZE:  (325604, 7)
DIFFERENCE:  0
 
DATAFRAME NAME:  coordinates_gis_osm_places_a_free_1.csv
ORIGINAL DATAFRAME SIZE:  (678, 7)
FILTERED DATAFRAME SIZE:  (88, 8)
DIFFERENCE:  590
 
DATAFRAME NAME:  coordinates_gis_osm_places_free_1.csv
ORIGINAL DATAFRAME SIZE:  (34542, 7)
FILTERED DATAFRAME SIZE:  (4778, 8)
DIFFERENCE:  29764
 
DATAFRAME NAME:  coordinates_gis_osm_pofw_a_free_1.csv
ORIGINAL DATAFRAME SIZE:  (11847, 6)
FILTERE

In [14]:
completed_df_dict = {}

for df_name, df in filtered_df_dict.items():

    if df.shape[0] > 0:
        # completed_df_dict[df_name] = df[~df.name.isna()]
        completed_df_dict[df_name] = df

In [15]:
for df_name in completed_df_dict.keys():
    print('DATAFRAME NAME: ', df_name)
    print('ORIGINAL DATAFRAME SIZE: ', df_dict[df_name].shape)
    print('FILTERED DATAFRAME SIZE: ', filtered_df_dict[df_name].shape)
    print('COMPLETE DATAFRAME SIZE: ', completed_df_dict[df_name].shape)
    print('DIFFERENCE: ', df_dict[df_name].shape[0] - completed_df_dict[df_name].shape[0])
    print(' ')

DATAFRAME NAME:  coordinates_gis_osm_landuse_a_free_1.csv
ORIGINAL DATAFRAME SIZE:  (536191, 6)
FILTERED DATAFRAME SIZE:  (394968, 7)
COMPLETE DATAFRAME SIZE:  (394968, 7)
DIFFERENCE:  141223
 
DATAFRAME NAME:  coordinates_gis_osm_natural_a_free_1.csv
ORIGINAL DATAFRAME SIZE:  (1205, 6)
FILTERED DATAFRAME SIZE:  (1205, 7)
COMPLETE DATAFRAME SIZE:  (1205, 7)
DIFFERENCE:  0
 
DATAFRAME NAME:  coordinates_gis_osm_natural_free_1.csv
ORIGINAL DATAFRAME SIZE:  (325604, 6)
FILTERED DATAFRAME SIZE:  (325604, 7)
COMPLETE DATAFRAME SIZE:  (325604, 7)
DIFFERENCE:  0
 
DATAFRAME NAME:  coordinates_gis_osm_places_a_free_1.csv
ORIGINAL DATAFRAME SIZE:  (678, 7)
FILTERED DATAFRAME SIZE:  (88, 8)
COMPLETE DATAFRAME SIZE:  (88, 8)
DIFFERENCE:  590
 
DATAFRAME NAME:  coordinates_gis_osm_places_free_1.csv
ORIGINAL DATAFRAME SIZE:  (34542, 7)
FILTERED DATAFRAME SIZE:  (4778, 8)
COMPLETE DATAFRAME SIZE:  (4778, 8)
DIFFERENCE:  29764
 
DATAFRAME NAME:  coordinates_gis_osm_pofw_a_free_1.csv
ORIGINAL DATAFRAM

In [16]:
os.makedirs('filtered/', exist_ok=True)

for df_name, df in completed_df_dict.items():

    file_name = 'filtered_' + df_name[:-4] + '.csv'
    df.to_csv('filtered/' + file_name)

In [17]:
completed_df_dict['coordinates_gis_osm_water_a_free_1.csv']

,osm_id,code,fclass,name,lon,lat,SELECT
0,4045604,8202,water,NaN,12.488169,44.944659,nature
2,4477981,8202,water,NaN,12.351897,45.429216,nature
4,4479546,8202,water,Canale dei Lavraneri,12.317752,45.427250,nature
6,4617487,8200,water,Canale de Sacca San Biagio,12.311209,45.426765,nature
8,4710677,8200,water,NaN,12.260212,45.497561,nature
...,...,...,...,...,...,...,...
50077,1096835041,8200,water,NaN,12.803509,46.639118,nature
50079,14005865,8202,water,NaN,13.331644,46.064916,nature
50081,1097248505,8200,water,NaN,12.994492,46.622447,nature
50083,1097248506,8200,water,NaN,12.996450,46.622617,nature
